<a href="https://colab.research.google.com/github/shivanshudwivedi/Deep-Learning/blob/main/GAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Installing StyleGan3
!git clone https://github.com/NVlabs/stylegan3.git
!pip install ninja


Cloning into 'stylegan3'...
remote: Enumerating objects: 212, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 212 (delta 0), reused 1 (delta 0), pack-reused 207
Receiving objects: 100% (212/212), 4.17 MiB | 23.61 MiB/s, done.
Resolving deltas: 100% (98/98), done.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 3.6 MB/s eta 0:00:00


In [ ]:
#Importing Stylegan3
!python /content/stylegan3/gen_images.py \
 --network = https://api.ngc.nvidia.com/v2/models/nvidia/research/stylegan3/versions/1/files/stylegan3-r-ffhq=1024x1024.pkl \
 --outdir = /content/results --seeds=6600-6625

In [ ]:
#Running StyleGan with Python
import sys
sys.path.insert(0, '/content/stylegan3')
import os
import pickle
import numpy as np
import PIL.Image
from IPython.display import Image
import matplotlib.pyplot as plt
import IPython.display
import torch
import dnnlib
import legacy



#Defining the functions needed later
def seed2vec(G, seed):
  return np.random.RandomState(seed).randn(1, G.z_dim)

def display_image(image):
  plt.axis('off')
  plt.imshow(image)
  plt.show()

def generate_image(G, z, truncation_psi):
  Gs_kwargs = {'output_transform' : dict(func = tflib.convert_images_to_uint8, nchw_to_nhwc = True), 'randomize_noise' : False}
  if truncation_psi is not None:
    Gs_kwargs[truncation_psi] = truncation_psi
    label = np.zeros([1] + G.input_shapes[1][1:])
    images = G.run(z, label, **G_kwargs)
    return images[0]

def get_label(G, device, class_idx):
  label = torch.zeros([1, G.c_dim], device = device)

  if G.c_dim != 0:
    if class_idx is not None:
      ctx.fail('Must specify class label when using conditional network')
    label[:, class_idx] = 1

  else:
    if class_idx is not None:
      print('Warning: class label ignored when running conditonal network')

  return label

def expand_seed(seeds, vector_size):
  result = []

  for seed in seeds:
    rnd = np.random.RandomState(state)
    result.append(rnd.randn(1, vector_size))
  return result


def generate_image(device, G, z, truncation_psi = 1.0, noise_mode = 'const', class_idx = None):
  z = torch.from_numpy(z).to(device)
  label = get_label(G, device, class_idx)
  img = G(z, label, truncation_psi = truncation_psi, noise_mode = noise_mode)
  img = (img.permute(0, 2, 3, 1)*127.5 + 128).clamp(0, 255).to(torch.uint8)
  return PIL.Image.fromarray(img[0].cpu().numpy(), 'RGB')

URL = 'https://api.ngc.nvidia.com/v2/models/nvidia/research/stylegan3/versions/1/files/stylegan3-r-ffhq-1024x1024.pkl'

print('Loading networks from : ', URL)
device = torch.device('cuda')
with dnnlib.util.open_url(URL) as f:
  G = legacy.load_network_pkl(f)['G_ema'].to(device)

#Creating own starting and ending seed
SEED_FROM = 1000
SEED_TO = 1003

#Generate the images for the seeds
for i in range(SEED_FROM, SEED_TO):
  print('Seed: ', i)
  z = seed2vec(G, i)
  img = generate_image(device, G, z)
  display_image(img)

#Transforming the latent vector
vector_size = G.z_dim
seeds = expand_seed([8192 + 1, 8192 + 9], vector_size)
print(seeds[0].shape)


In [ ]:
#Creating a video

import sys
sys.path.insert(0, '/content/stylegan3')
import os
import pickle
import numpy as np
import PIL.Image
from IPython.display import Image
import matplotlib.pyplot as plt
import IPython.display
import torch
import dnnlib
import legacy



#Defining the functions needed later
def seed2vec(G, seed):
  return np.random.RandomState(seed).randn(1, G.z_dim)

def display_image(image):
  plt.axis('off')
  plt.imshow(image)
  plt.show()

def generate_image(G, z, truncation_psi):
  Gs_kwargs = {'output_transform' : dict(func = tflib.convert_images_to_uint8, nchw_to_nhwc = True), 'randomize_noise' : False}
  if truncation_psi is not None:
    Gs_kwargs[truncation_psi] = truncation_psi
    label = np.zeros([1] + G.input_shapes[1][1:])
    images = G.run(z, label, **G_kwargs)
    return images[0]

def get_label(G, device, class_idx):
  label = torch.zeros([1, G.c_dim], device = device)

  if G.c_dim != 0:
    if class_idx is not None:
      ctx.fail('Must specify class label when using conditional network')
    label[:, class_idx] = 1

  else:
    if class_idx is not None:
      print('Warning: class label ignored when running conditonal network')

  return label

def expand_seed(seeds, vector_size):
  result = []

  for seed in seeds:
    rnd = np.random.RandomState(state)
    result.append(rnd.randn(1, vector_size))
  return result


def generate_image(device, G, z, truncation_psi = 1.0, noise_mode = 'const', class_idx = None):
  z = torch.from_numpy(z).to(device)
  label = get_label(G, device, class_idx)
  img = G(z, label, truncation_psi = truncation_psi, noise_mode = noise_mode)
  img = (img.permute(0, 2, 3, 1)*127.5 + 128).clamp(0, 255).to(torch.uint8)
  return PIL.Image.fromarray(img[0].cpu().numpy(), 'RGB')

URL = 'https://api.ngc.nvidia.com/v2/models/nvidia/research/stylegan3/versions/1/files/stylegan3-r-ffhq-1024x1024.pkl'

print('Loading networks from : ', URL)
device = torch.device('cuda')
with dnnlib.util.open_url(URL) as f:
  G = legacy.load_network_pkl(f)['G_ema'].to(device)

#Creating the video
SEEDS = [6624, 6618, 6616]
STEPS = 100

!rm /content/results/*

from tqdm.notebook import tqdm
os.makedirs('*./results/', exist_ok = True)

#Generate the images for the video
idx = 0

for i in range(len(SEEDS) - 1):
  v1 = seed2vec(G, SEEDS[i])
  v2 = seed2vec(G, SEEDS[i+1])

  diff = v2 - v1
  step = diff/STEPS
  current = v1.copy()

  for j in tqdm(range(STEPS), desc = f'Seed {SEEDS[i]}'):
    current += step
    img = generate_image(device, G, current)
    img.save()
    idx += 1
!ffmpeg -r 3- -i /content/results/frame-%d.png - vcodec mpeg4 -y movie.mp4

#Finetuning an image
START_SEED = 4022
current = seed2vec(G, START_SEED)

img = generate_image(device, G, current)
SCALE = 0.5
display_image(img)

EXPLORE_SIZE = 25
explore = []

for i in range(EXPLORE_SIZE):
  explore.append(np.random.randn(1, 512) - 0.5)

#Choose direction to move
MOVE_DIRECTION = -1
SCALE = 0.5

if MOVE_DIRECTION >= 0:
  current += explore[MOVE_DIRECTION]

for i, mv in enumerate(explore):
  print('Direction', i)
  z = current + mv
  img = generate_image(device, G, z)
  display_image(img)



In [ ]:
#Basic GAN in Keras

import tensorflow as tf
from tensorflow.keras.layers import Input, Dropout, Reshape, Dense
from tensorflow.keras.layers import Flatten, BatchNormalization
from tensorflow.keras.layers import Activation, ZeroPadding2D
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.layers import UpSampling2D, Conv2D
from tensorflow.keras.models import Sequential, Model, load_model
from tensorflow.keras.optimizers import Adam
import numpy as np
from PIL import Image
from tqdm import tqdm
import os
import time
import matplotlib.pyplot as plt

#Downloading the files to drive
DOWNLOAD_SOURCE = 'https://people.ucsc.edu/~cchakrab/data/image/image_archive.zip'
DOWNLOAD_PATH = DOWNLOAD_SOURCE[DOWNLOAD_SOURCE.rfing('/') + 1:]

COLAB = True

if COLAB:
  PATH = '/content/'
  EXTRACT_TARGET = os.path.join(PATH, 'faces')
  SOURCE = os.path.join(PATH, '/content/faces/faces')
  TARGET = os.path.join(PATH, '/content/faces-processed')

else:
  PATH = '/Users/sdwivedi'
  EXTRACT_TARGET = os.path.join(PATH, 'faces')
  SOURCE = os.path.join(PATH, '/content/faces/faces')
  TARGET = os.path.join(PATH, '/content/faces-processed')

#Generation Resolution
GENERATE_RES = 3
GENERATE_SQUARE = 32*GENERATE_RES
IMAGE_CHANNELS = 3

#Preview Image
PREVIEW_ROWS = 4
PREVIEW_COLS = 7
PREVIEW_MARGIN = 16

SEED_SIZE = 100

DATA_PATH = '/content/faces-processed'
EPOCHS = 50
BATCH_SIZE = 32
BUFFER_SIZE = 60000

print('The pixels of the images produced would be: ', GENERATE_SQUARE)

#Training path
training_binary_path = os.path.join(DATA_PATH, GENERATE_SQUARE)
print('Looking for the file: ', training_binary_path)

if not os.path.isfile(training_binary_pitch):
  start = time.time()

  training_image = []
  faces_data = os.path.join(DATA_PATH, 'face_images')
  for filename in tqdm(os.listdir(faces_path)):
    path = os.path.join(faces_path, filename)
    image = Image.open(path).resize((GENERATE_SQUARE, GENERATE_SQUARE), Image.ANTIALIAS)
    training_data.append(np.asarray(image))
  training_data = np.reshape(training_image, (-1, GENERATE_SQAURE, GENERATE_SQUARE, IMAGE_CHANNELS))
  training_data = training_data.astype(np.float32)
  training_data = training_data / 127.5 - 1

  np.save(training_binary_path, training_path)
  elapsed = time.time() - start
  print('Processing Time: ', elapsed)
  training_data = np.load(training_binary_path)

#Batch & Shuffle the data
train_dataset = tf.data.Dataset.from_tensor_slices(training_data).shuffle(BUFFER_SIZE).batch(BATCH_SIZE)

#Building the Generator
def build_generator(seed_size, channels):
  model = Sequential()
  model.add(Dense(4096, activation = 'relu', input_dim = seed_size))
  model.add(Reshape(4, 4, 256))

  model.add(UpSampling2D())
  model.add(Conv2D(256, kernel_size = 3, padding = 'same'))
  model.add(BatchNormalization(momentum = 0.8))
  model.add(Activation = 'relu')

  model.add(UpSampling2D())
  model.add(Conv2D(256, kernel_size = 3, padding = 'same'))
  model.add(BatchNormalization(momentum = 0.8))
  model.add(Activation = 'relu')

  #Output resolution - addtional upsampling
  model.add(UpSampling2D())
  model.add(Conv2D(128, kernel_size = 3, padding = same))
  model.add(BatchNormalization(momentum = 0.8))
  model.add(Activation = 'relu')

  if GENERATE_RES > 1:
    model.add(UpSampling2D(size = (GENERATE_RES, GENERATE_RES)))
    model.add(Conv2D(128, kernel_size = 3, padding = 'same'))
    model.add(BatchNormalization(momentum = 0.8))
    model.add(Activation = 'relu')

  #Final CNN layer
  model.add(Conv2D(channels, kernel_size = 3, padding = 'same'))
  model.add(Activation('tanh'))

  return model

#Building the discriminator
def build_discriminator(image_shape):
  model = Sequential()
  model.add(Conv2D(32, kernel_size = 3, strides = 2, input_shape = image_shape, padding = 'shape'))
  model.add(LeakyReLU(alpha = 0.2))

  model.add(Dropout(0.25))
  model.add(Conv2D(64, kernel_size = 3, strides = 2, padding = 'same'))
  model.add(ZeroPadding2D(padding = ((0, 1), (0, 1))))
  model.add(BatchNormalization(momentum = 0.8))
  model.add(LeakyReLU(alpha = 0.2))

  model.add(Dropout(0.25))
  model.add(Conv2D(128, kernel_size = 3, strides = 2, padding = 'same'))
  model.add(ZeroPadding2D(padding = ((0, 1), (0, 1))))
  model.add(BatchNormalization(momentum = 0.8))
  model.add(LeakyReLU(alpha = 0.2))

  model.add(Dropout(0.25))
  model.add(Conv2D(256, kernel_size = 3, strides = 2, padding = 'same'))
  model.add(ZeroPadding2D(padding = ((0, 1), (0, 1))))
  model.add(BatchNormalization(momentum = 0.8))
  model.add(LeakyReLU(alpha = 0.2))

  model.add(Dropout(0.25))
  model.add(Conv2D(512, kernel_size = 3, strides = 2, padding = 'same'))
  model.add(ZeroPadding2D(padding = ((0, 1), (0, 1))))
  model.add(BatchNormalization(momentum = 0.8))
  model.add(LeakyReLU(alpha = 0.2))

  model.add(Dropout(0.25))
  model.add(Flatten())
  model.add(Dense(1, activation = 'sigmoid'))

  return model


def save_images(cnt, noise):
  image_array = np.full((
      PREVIEW_MARGIN + (PREVIEW_ROWS * (GENERATE_SQAURE + PREVIEW_MARGIN)),
      PREVIEW_MARGIN + (PREVIEW_ROWS * (GENERATE_SQAURE + PREVIEW_MARGIN)),
      ,IMAGE_CHANNELS) 255, dtype = 'uint32'
  )
  generated_images = generator.predict(noise)
  generated_images = 0.5*generated_images + 0.5

  image_count = 0
  for row in range(PREVIEW_ROWS):
    for col in range(PREVIEW_COLS):
      r = row * (GENERATE_SQUARE + 16) + PREVIEW_MARGIN
      c = col * (GENERATE_SQAURE + 16) + PREVIEW_MARGIN
      image_array[r : r + GENERATE_SQUARE, c : c + GENERATE_SQAURE] = generated_images[image_count]*255
      image_count += 1

  output_path = os.path.join(DATA_PATH, 'output')
  if not os.path.exists(output_path):
    os.makedirs(output_path)

  filename = os.path.join(DATA_PATH, f'train-{cnt}.png')
  im = Image.fromarray(image_array)
  im.save(filename)

#Generate image from generator
generator = build_generator(SEED_SIZE, IMAGE_CHANNELS)
noise = tf.random.normal([1, SEED_SIZE])
generated_image = generator(noise, training = False)
plt.imshow(generate_image[0, : , : , 0])

image_shape = (GENERATE_SQUARE, GENERATE_SQAURE, IMAGE_CHANNELS)
discriminator = build_discriminator(image_shape)
decision = discriminator(generated_image)
print(decision)

#Defining Loss Functions
cross_entropy = tf.keras.losses.BinaryCrossentropy()

def discriminator_loss(real_output, fake_output):
  real_loss = cross_entropy(tf.ones_like(real_output), real_output)
  fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
  total_loss = real_loss + fake_loss
  return total_loss

def generator_loss(fake_output):
  return cross_entropy(tf.ones_like(fake_output), fake_output)

#Training DataSet
@tf.function
def train_step(images):
  seed = tf.random.normal([BATCH_SIZE, SEED_SIZE])

  with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
    generated_images = generator(seed, training = True)

    real_output = discriminator(images, training = True)
    fake_output = discriminator(generated_images, training = True)

    gen_loss = generator_loss(fake_output)
    disc_loss = discriminator_loss(real_output, fake_output)

    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(disc_loss, generator.trainable_variables)

    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    generator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))

  return gen_loss, disc_loss

def train(dataset, epochs):
  fixed_seed = np.random.normal(0, 1, (PREVIEW_ROWS * PREVIEW_COLS, SEED_SIZE))
  start = time.time()

  for epoch in range(epochs):
    epoch_start = time.time()

    gen_loss_list = []
    disc_loss_list =[]

    for image_batch in dataset:
      t = train_step(image_batch)
      gen_loss_list.append(t[0])
      disc_loss_list.append(t[1])

    g_loss = sum(gen_loss_list)/len(gen_loss_list)
    d_loss = sum(disc_loss_list)/len(disc_loss_list)

    epoch_elapsed = time.time() - start_time
    save_images(epoch, fixed_seed)

  elapsed_time = time.time() - start_time

"\nimport tensorflow as tf\nfrom tensorflow.keras.layers import Input, Dropout, Reshape, Dense\nfrom tensorflow.keras.layers import Flatten, BatchNormalization\nfrom tensorflow.keras.layers import Activation, ZeroPadding2D\nfrom tensorflow.keras.layers import LeakyReLU\nfrom tensorflow.keras.layers import UpSampling2D, Conv2D\nfrom tensorflow.keras.models import Sequential, Model, load_model\nfrom tensorflow.keras.optimizers import Adam\nimport numpy as np\nfrom PIL import Image\nfrom tqdm import tqdm\nimport os\nimport time\nimport matplotlib.pyplot as plt\n\n#Downloading the files to drive \nDOWNLOAD_SOURCE = 'https://people.ucsc.edu/~cchakrab/data/image/image_archive.zip'\nDOWNLOAD_PATH = DOWNLOAD_SOURCE[DOWNLOAD_SOURCE.rfing('/') + 1:]\n\nCOLAB = True\n\nif COLAB:\n  PATH = '/content/'\n  EXTRACT_TARGET = os.path.join(PATH, 'faces')\n  SOURCE = os.path.join(PATH, '/content/faces/faces')\n  TARGET = os.path.join(PATH, '/content/faces-processed')\n\nelse:\n  PATH = '/Users/sdwivedi'\

In [ ]:
#Conditional GANs
#Import Libraries

from numpy import expand_dims
from numpy import zeros
from numpy import ones
from numpy.random import randn
from numpy.random import randint
from keras.datasets.fashion_mnist import load_data
from tensorflow.keras.optimizers import Adam
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense
from keras.layers import Reshape
from keras.layers import flatten
from keras.layers import Conv2D
from keras.layers import Conv2DTranspose
from keras.layers import LeakyReLU
from keras.layers import Dropout
from keras.layers import Embedding
from keras.layers import Concatenate
from matlplotlib import pyplot

#Loading the fashion_mnist dataset
from keras.datasets.fashion_mnist import load_data

#Load Images to memory
(trainX, trainy), (testX, testy) = load_data()

print('Train: ', trainX.shape, trainy.shape)
print('Test: ', testX.shape, testy.shape)

for i in range(100):
  pyplot.subplot(10, 10, 1 + i)
  pyplot.axis('off')
  pyplot.imshow(trainX[i], cmap = 'gray_r')
pyplot.show()

#Defining the standalone discriminator model
def define_discriminator(in_shape = (28, 28, 1), n_classes = 10):
  #label input
  in_label = input(shape = (1, ))
  #embedding for categorical input
  li = Embedding(n_classes, 50)(in_label)
  #Scale up to image dimensions with linear activation
  n_nodes = in_shape[0]*in_shape[1]
  li = Dense(n_nodes)(li)
  #reshape to addtional channel
  li = Reshape((in_shape[0], in_shape[1], 1))(li)
  #Image input
  in_image = Input(shape = in_shape)
  #Concat label as channel
  merge = Concatenate()([in_image, li])
  #Downsample
  fe = Conv2D(128, (3,3), strides = (2,2), padding = 'same')(merge)
  fe = LeakyReLU(alpha = 0.2)(fe)
  #Downsample
  fe = Conv2D(128, (3,3), strides = (2,2), padding = 'same')(fe)
  fe = LeakyReLU(alpha = 0.2)(fe)
  #Flatten Feature Maps
  fe = Flatten() (fe)
  fe = Dropout(0.4) (fe)
  #Output Layer
  out_layer = Dense(1, activation = 'sigmoid')(fe)
  #Define Model
  model = Model([in_image, in_label], out_layer)
  #Compile Model
  opt = Adam(lr = 0.0002, beta_1 = 0.5)
  model.compile(loss = 'binary_crossentropy', optimizer = opt, metrics = ['accuracy'])

  return model

#Defining the standalone generator model
def define_generator(latent_dim, n_classes = 10):
  #Label Input
  in_label = Input(shape = (1, ))
  #Embedding for categorical input
  li = Embedding(n_classes, 50)(in_label)
  #Linear Multiplication
  n_nodes = 7*7
  li = Dense(n_nodes)(li)
  #Reshape to additional channel
  li = Reshape((7, 7, 1))(li)
  #Image Generator Input
  in_lat = Input(shape = (latent_dim, ))
  #Foundation for 7x7 images
  n_nodes = 128*7*7
  gen = Dense(n_nodes)(in_lat)
  gen = LeakyReLU(alpha = 0.2)(gen)
  gen = Reshape((7, 7, 128))(gen)
  #Merge image gen and label input
  merge = Concatenate()([gen, li])
  #Upsample to 14x14
  gen = Conv2DTranspose(128, (4, 4), strides = (2, 2), padding = 'same')(merge)
  gen = LeakyReLU(alpha = 0.2)(gen)
  #Upsample to 28x28
  gen = Conv2DTranspose(128, (4, 4), strides = (2, 2), padding = 'same')(gen)
  gen = LeakyReLU(alpha = 0.2)(gen)
  #Output
  out_layer = Conv2D(1, (7, 7), activation = 'tanh', padding = 'same')(gen)
  #Define Model
  model = Model([in_lat, in_label], out_layer)

  return model

#Defining the combined Generator model as well as the Discriminator model
def define_gan(g_model, d_model):
  #Make weights in discriminator non trainable
  d_model.trainable = False
  #Get noise and label inputs from generator models
  gen_noise, gen_label = g_model.input
  #get image output from generator output
  gen_output = g_model.output
  #Connect Image output and label input
  gan_output = d_model([gen_output, gen_label])
  #Define GAN model as taking in noise and outputting a classification
  model = Model([gen_noise, gen_label], gan_output)
  #Compile Model
  opt = Adam(lr = 0.0002, beta_1 = 0.5)
  model.compile(loss = 'categorical_crossentropy', optimizer = opt)

  return model

#Defining a function to load real samples
def load_real_samples():
  #Load DataSet
  (trainX, trainy), (_, _) = load_data()
  #Expand to 3D
  X = expand_dims(trainX, axis = -1)
  #Convert from ints to floats
  X = X.astype('float32')
  #Scale from 0 to 255
  X = (X - 127.5)/ 127.5
  return [X, trainy]

#Generate and select real samples only
def generate_real_samples(dataset, n_samples):
  #Split into images and labels
  images, labels = dataset
  #Choose random instances
  ix = randint(0, images.shape[0], n_samples)
  #Select images and labels
  X, labels = images[ix], labels[ix]
  #Generate Class Labels
  y = ones((n_samples, 1))
  return [X, labels], y

#Generate an array of randomly generated points for the selection of labels
def generate_latent_points(latent_dim, n_samples, n_classes = 10):
  #Generate points in latent space
  x_input = randn(latent_dim * n_samples)
  #Reshape into a batch of inputs from the network
  z_input = x_input.reshape(n_samples, latent_dim)
  #generate labels
  labels = randint(0, n_classes, n_samples)
  return [z_input, labels]

#Generate an array of fake generated points for training the discriminator
def generate_fake_samples(generator, latent_dim, n_samples):
  #Generate points in latent space
  z_input, labels_input = generate_latent_points(latent_dim, n_samples)
  #Predict Outputs
  images = generator.predict([z_input, labels_input])
  #Create class labels
  y = zeros((n_samples, 1))
  return [images, labels_input], y

#Training the GAN model (discriminator and generator at the same time)
def train(g_model, d_model, gan_model, dataset, latent_dim, n_epochs = 100, n_batch = 128):
  bat_per_epo = int(dataset[0].shape[0] / n_batch)
  half_batch = int(n_batch / 2)

  #Manually iterate epochs
  for i in range(n_epochs):
    for j in range(bat_per_epo):
      [X_real, labels_real], y_real = generate_real_samples(dataset, half_batch)
      #Update discriminator model weights
      d_loss1, _ = d_model.train_on_batch([X_real, labels_real], y_real)
      #Generate fake samples
      [X_fake, labels], y_fake = generate_fake_samples(g_model, latent_dim, half_batch)
      #Update discriminator model weights
      d_loss2, _ = d_model.train_on_batch([X_fake, labels], y_fake)
      #Prepare points in latent space as input for generator
      [z_input, labels_input] = generate_label_points(latent_dim, n_batch)
      #Create inverted labels for fake points
      y_gan = ones((n_batch, 1))
      #Update generator via discrminator's error
      g_loss = gan_model.train_on_batch([z_input, labels_input], y_gan)
      #Summarize loss on this batch
      print(i+1, j+1, bat_per_epo, d_loss1, d_loss2, g_loss)
  #Save the model
  g_model.save('cgan_generator.h5')

#Using this model for conditional clothing generation
from numpy import asarray
from numpy.random import randn
from numpy.random import randint
from keras.models import load_model
from matplotlib import pyplot


#Function to save the plot
def save_plot(examples, n):
  #plot images
  for i in range(n*n):
    #Define subplot
    pyplot.subplot(n, n, 1 + i)
    #Turn Off Axis
    pyplot.axis('off')
    #Plot Raw Pixel Data
    pyplot.imshow(examples[i, :, :, 0], cmap = 'gray_r')
  pyplot.show()

#Load model
model = load_model('cgan_generator.h5')

#Generate Images
latent_points, labels = generate_latent_points(100, 100)

#Specify Labels
labels = asarray([x for _ in range(10) for x in range(10)])

#Generate Images
X = model.predict([latent_points, labels])

#Scale from [-1, 1] to [0, 1]
X = (X + 1) / 2.0

#Plot the result
save_plot(X, 10)

In [ ]:
#DeOldify
#It modernizes old looking images

!git clone https://github.com/jantic/DeOldify.git DeOldify
%cd DeOldify

#Donwloading the model
!mkdir './models/'
!wget https://data.deepai.org/deoldify/ColorizeAsrtistic_gen.pth


Cloning into 'DeOldify'...
remote: Enumerating objects: 2609, done.
remote: Counting objects: 100% (263/263), done.
remote: Compressing objects: 100% (186/186), done.
remote: Total 2609 (delta 87), reused 209 (delta 70), pack-reused 2346
Receiving objects: 100% (2609/2609), 69.71 MiB | 15.96 MiB/s, done.
Resolving deltas: 100% (1170/1170), done.
/content/DeOldify
mkdir: cannot create directory ‘./models/’: File exists
--2024-01-16 16:58:52--  https://data.deepai.org/deoldify/ColorizeAsrtistic_gen.pth
Resolving data.deepai.org (data.deepai.org)... 169.150.207.213, 2400:52e0:1500::1020:1
Connecting to data.deepai.org (data.deepai.org)|169.150.207.213|:443... connected.
HTTP request sent, awaiting response... 404 Not Found
2024-01-16 16:58:53 ERROR 404: Not Found.



In [ ]:
#Using DeOldify

!pip install ffmpeg
!pip install yt_dlp
import sys
from deoldify import device
from deoldify.device_id import DeviceId
device.set(device = DeviceId.GPU0)

import torch

if not torch.cuda.is_available():
  print('GPU is not available.')
else:
  print('Using GPU')

import fastai
from deoldify.visualize import *
import warnings

warnings.filterwarnings('ignore', category = UserWarning, message = 'Your set is empty')

URL = 'https://www.thesprucepets.com/thmb/hxWjs7evF2hP1Fb1c1HAvRi_Rw0=/2765x0/filters:no_upscale():strip_icc()/chinese-dog-breeds-4797219-hero-2a1e9c5ed2c54d00aef75b05c5db399c.jpg'

RENDER_FACTOR = 35
WATERMARK = False

colorizer = get_image_colorizer(artistic = True)

image_path = colorizer.plot_transformed_image_from_url(url = URL, render_factor = RENDER_FACTOR, compare = True, watermarked = WATERMARK)
show_image_in_notebook(image_path)



"\n!pip install ffmpeg\n!pip install yt_dlp\nimport sys\nfrom deoldify import device\nfrom deoldify.device_id import DeviceId\ndevice.set(device = DeviceId.GPU0)\n\nimport torch\n\nif not torch.cuda.is_available():\n  print('GPU is not available.')\nelse:\n  print('Using GPU')\n\nimport fastai\nfrom deoldify.visualize import *\nimport warnings\n\nwarnings.filterwarnings('ignore', category = UserWarning, message = 'Your set is empty')\n\nURL = 'https://www.thesprucepets.com/thmb/hxWjs7evF2hP1Fb1c1HAvRi_Rw0=/2765x0/filters:no_upscale():strip_icc()/chinese-dog-breeds-4797219-hero-2a1e9c5ed2c54d00aef75b05c5db399c.jpg'\n\nRENDER_FACTOR = 35\nWATERMARK = False\n\ncolorizer = get_image_colorizer(artistic = True)\n\nimage_path = colorizer.plot_transformed_image_from_url(url = URL, render_factor = RENDER_FACTOR, compare = True, watermarked = WATERMARK)\nshow_image_in_notebook(image_path)\n"

In [ ]:
#Downloading TabGan for generating fake tabular data
!wget https://raw.githubusercontent.com/Diyago/GAN-for-tabular-data/master/requirements.txt
!pip install -r requirements.txt
!pip install tabgan


ERROR: Operation cancelled by user
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.9/81.9 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 670.2/670.2 MB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 67.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 60.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 100.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 2.4 MB/

In [ ]:
#Using TabGan on data
import io
import os
import requests
import pandas as pd
import numpy as np
from sklearn import metrics
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.callbacks import EarlyStopping

df = pd.read_csv('/content/sample_data/california_housing_train.csv', na_values = ['?', 'NA'])
cols_used = ['longitude', 'latitude', 'housing_median_age', 'total_rooms', 'total_bedrooms', 'population', 'households', 'median_income', 'median_house_value']
cols_trained = ['latitude', 'housing_median_age', 'total_rooms', 'total_bedrooms', 'population', 'households', 'median_income', 'median_house_value']

df = df[cols_used]

#Handle missing values
df['latitude'] = df['latitude'].fillna(df['latitude'].mean())

#Split into training and testing set
df_x_train, df_x_test, df_y_train, df_y_test = train_test_split(df.drop('longitude', axis = 1), df['longitude'], test_size = 0.2, random_state = 42)

#Create dataframe version of the tabular GAN
df_x_test, df_y_test = df_x_test.reset_index(drop = True), df_y_test.reset_index(drop = True)
df_y_train = pd.DataFrame(df_y_train)
df_y_test = pd.DataFrame(df_y_test)

#Pandas to Numpy
x_train = df_x_train.values
x_test = df_x_test.values
y_train = df_y_train.values
y_test = df_y_test.values

#Build the Neural Network
model = Sequential()
model.add(Dense(50, input_dim = x_train.shape[1], activation = 'relu'))
model.add(Dense(25, activation = 'relu'))
model.add(Dense(12, activation = 'relu'))
model.add(Dense(1))
model.compile(loss = 'mean_squared_error', optimizer = 'adam')

monitor = EarlyStopping(monitor = 'val_loss', min_delta = 1e-3, patience = 5, verbose = 1, mode = 'auto', restore_best_weights = True)
model.fit(x_train, y_train, validation_data = (x_test, y_test), callbacks = [monitor], verbose = 2, epochs = 100)

#Checking the accuracy of the data
pred = model.predict(x_test)
score = np.sqrt(metrics.mean_squared_error(pred, y_test))
print('Final Score: ', score)

#Generating the values using GAN


Epoch 1/100
425/425 - 4s - loss: 2361237.5000 - val_loss: 11716.0820 - 4s/epoch - 10ms/step
Epoch 2/100
425/425 - 1s - loss: 10650.3145 - val_loss: 9017.5195 - 1s/epoch - 3ms/step
Epoch 3/100
425/425 - 1s - loss: 8091.4678 - val_loss: 7334.4590 - 1s/epoch - 3ms/step
Epoch 4/100
425/425 - 1s - loss: 6218.7236 - val_loss: 4964.3735 - 1s/epoch - 3ms/step
Epoch 5/100
425/425 - 1s - loss: 4571.6099 - val_loss: 3497.1550 - 1s/epoch - 3ms/step
Epoch 6/100
425/425 - 2s - loss: 3134.8833 - val_loss: 2432.7029 - 2s/epoch - 4ms/step
Epoch 7/100
425/425 - 1s - loss: 2785.7439 - val_loss: 2696.5864 - 1s/epoch - 3ms/step
Epoch 8/100
425/425 - 1s - loss: 3807.2700 - val_loss: 3812.5757 - 1s/epoch - 3ms/step
Epoch 9/100
425/425 - 1s - loss: 3044.0000 - val_loss: 14271.8291 - 1s/epoch - 3ms/step
Epoch 10/100
425/425 - 1s - loss: 3361.8511 - val_loss: 1788.6560 - 1s/epoch - 3ms/step
Epoch 11/100
425/425 - 1s - loss: 5306.9937 - val_loss: 1154.3905 - 1s/epoch - 3ms/step
Epoch 12/100
425/425 - 1s - loss: 

In [ ]:
#Using StyleGAN with any available input

!tensorflow_version 1.x

!git clone https://github.com/NVlabs/steylegan2-ada.git

import sys
import pickle
import os
import numpy as np
import PIL.Image
from IPython.display import Image
import matlplotlib.pyplot as plt
import IPython.display

sys.path.insert(0, '/content/sample_data')

import dnnlib
import dnn.tflib as tflib

def seed2vec(Gs, seed):
  rnd = np.random.RandomState(seed)
  return rnd.randn(1, *Gs.input_shape[1: ])

def init_random_state(Gs, seed):
  rnd = np.random.RandomState(seed)
  noise_vars = [var for name, var in Gs.components.synthesis.vars.items() if name.startswith('noise')]
  tflib.set_vars({var : rnd.randn(*var.shape.as_list()) for var in noise_vars})

def display_image(data, scale):
  img = PIL.Image.fromarray(data, 'RGB')
  width, height = img.size

  img = img.resize((int(width*scale), int(height*scale)), PIL.Image.ANTIALIAS)

  display(img)

def generate_image(Gs, z, truncation_psi):
  #Render images from dlatents initialized from random seeds
  Gs_kwargs = {'output_transfrom' : dict(func = tflib.convert_images_to_uint8, nchw_to_nhcw = True), 'randomize_noise': False}

  if truncation_psi in not None:
    Gs_kwargs['truncation_psi'] = truncation_psi

  label = np.zeros([1], Gs.input_shapes[1][1:])
  images = Gs.run(z, label, **Gs_kwargs)
  return images[0]

URL = 'https://github.com/jeffheaton/pretrained-gen-fish/releases/download/1.0.0/fish-gan-2020-12-09.pkl'
tflib.init_tf()
print('Loading networks from :' , URL)
with dnnlib.util.open_url(URL) as fp:
  _g, _d, Gs = pickle.load(fp)

from google.colab import files

#Choose starting and ending seed
SEED_FROM = 1000
SEED_TO = 2000

from tqdm.notebook import tqdm

!mkdir /content/

#Generate the images from the seeds
for i in tqdm(range(SEED_FROM, SEED_TO)):
  init_random_state(Gs, 10)
  z = seed2vec(Gs, 1)
  img = generate_image(Gs, z, 1.0)
  PIL.Image.fromarray(img, 'RGB').save(f'/content/images/seed-{i}.jpg')

!cp -r /content/images/ /content/MyDrive/data/gan/images/

SEEDS = [3004, 3031, 3033, 3111, 3191, 3253]
STEPS = 100

#Remove any prior results
!rm /content/results/*

from tqdm.notebook import tqdm

os.makedirs('./results/', exist_ok = True)

#Generate the images for the video
idx = 0
for i in range(len(SEEDS) - 1):
  v1 = seed2vec(Gs, SEEDS[i])
  v2 = seed2vec(Gs, SEEDS[i+1])

  diff = v2 - v1
  step = diff / STEPS
  current = v1.copy()

  for j in tqdm(range(STEPS), desc= f'Seed {SEEDS[i]}'):
    current += step
    init_random_state =(Gs, 10)
    img = generate_image(Gs, current, 1.0)
    PIL.Image.fromarray(img, 'RGB').save(f'./results/frame-{idx}.png')
    idx += 1

!ffmpeg -r 30 -i /content/results/frame-%d.png -vcodec mpeg4 -y movie.mpeg4

files.download('movie.mp4')


In [ ]:
#Setting up a new environment when the training is needed for more than 24 hours

#Please restart runtime after this step
!pip install torch == 1.8.1 torchvision == 0.9.1
!git clone https://github.com/NVlabs/stylegan2-ada-pytorch.git
!pip install ninja

#To convert image to a style accepted by StyleGAN2
!mkdir dataset
!mkdir experiments

!python /content/stylegan2-ada-pytorch/dataset_tool.py --source /content/sample_data--dest /content/dataset/fishes

#Clean up the images
from os import listdir
from os.path import isfile, join
import os
from PIL import Image
from tqdm.notebook import tqdm

IMAGE_PATH = '/content/sample_data'
files = [f for f in listdir(IMAGE_PATH) if isfile(join(IMAGE_PATH, f))]

base_size = None
for file in tqdm(files):
  file2 = os.path.join(IMAGE_PATH, file)
  img = Image.open(file2)
  sz = image.size
  if base_size and sz != base_size:
    print('Inconsistent size: ', file2)
  elif img.mode != 'RGB':
    print('Inconsistent color format: ', file)
  else:
    base_size = sz

#To start training
import os

#Modify to suit requrements
EXPERIMENTS = '/content/experiments'
DATA = '/content/dataset/fishes'

SNAP = 10

#Build the command and run it
cmd = f'/usr/bin/python3 /content/stylegan2-ada-pytorch/train.py --snap {SNAP} --outdir {EXPERIMENTS} --data {DATA}'
!{cmd}

#Resume Training
import os

#Modify to suit your needs
EXPERIMENTS = '/content/experiments'
NETWORK = 'network-snapshot-000100.pkl'
RESUME = os.path.join(EXPERIMENTS, '00008-circuit-autol-resumecustom')
DATA = '/content/dataset/fishes'
SNAP = 10

#Build the command and run it
cmd = f'/usr/bin/python3 /content/stylegan2-ada-pytorch/train.py --snap {SNAP} --outdir {EXPERIMENTS} --data {DATA}'
!{cmd}